## Download from RShiny

In [1]:
"""
Download LA County neighborhood testing data.
Download persons tested and tests performed.
Download from RShiny.
No date column, it probably reflects 2 days ago, given their comment in sidebar (community testing).
"""
import pandas as pd
import pytz

from datetime import datetime, timedelta

session_id = "_w_938e5948"
session_string = "a7404da85b714a061419f258a9a71f7b"
session_id2 = session_id[3:]

NEIGHBORHOOD_TESTING_URL = (
    f"https://lacdph.shinyapps.io/covid19_surveillance_dashboard/{session_id}/"
    f"session/{session_string}/download/download4?w={session_id2}"
)
  
TESTS_PERFORMED = (
    f"https://lacdph.shinyapps.io/covid19_surveillance_dashboard/{session_id}/"
    f"session/{session_string}/download/download6?w={session_id2}"
)

PERSONS_TESTED = (
    f"https://lacdph.shinyapps.io/covid19_surveillance_dashboard/{session_id}/"
    f"session/{session_string}/download/download5?w={session_id2}"   
)

S3_FILE_PATH = "s3://public-health-dashboard/jhu_covid19/"


today = datetime.today().astimezone(pytz.timezone("US/Pacific")).date()
#month = today.strftime('%m')
#day = today.strftime("%d")


# Neighborhood Testing data
def download_from_rshiny():
    df = pd.read_csv(NEIGHBORHOOD_TESTING_URL)

    df = (df.assign(
            download_date = today,
            date = today - timedelta(days=2)
        ).rename(columns = {"geo_merge": "neighborhood"})
    )

    df = (df.assign(
            download_date = pd.to_datetime(df.download_date),
            date = pd.to_datetime(df.date)
        ).drop(columns = "Unnamed: 0")
    )

    df.to_parquet(f"{S3_FILE_PATH}la-county-neighborhood-testing-{today}.parquet")

    
    tests_df = pd.read_csv(TESTS_PERFORMED)
    tests_df.to_csv(f"{S3_FILE_PATH}county-tests-performed-rshiny.csv")

    persons_df = pd.read_csv(PERSONS_TESTED)
    persons_df.to_csv(f"{S3_FILE_PATH}county-persons-tested-rshiny.csv")
    

download_from_rshiny()

## Append the testing by neighborhood

In [2]:
NEIGHBORHOOD_APPENDED = f"{S3_FILE_PATH}la-county-neighborhood-testing-appended.parquet"

# Append dfs
df = pd.read_parquet(NEIGHBORHOOD_APPENDED)

# Just append the single day to the appended df
day_df = pd.read_parquet(f"{S3_FILE_PATH}la-county-neighborhood-testing-{today}.parquet")
df = df.append(day_df).drop_duplicates()
    
df.to_parquet(NEIGHBORHOOD_APPENDED)

# At this point, this neighborhood URL can be read in neighborhood_utils
# The NEIGHBORHOOD_APPENDED file gets cleaned, cols dropped
# Run notebooks/neighborhood-charts after this, because the neighborhood_utils will write to parquet